In [1]:
import pandas as pd
import sklearn
import numpy as np

In [54]:
from sklearn.decomposition import PCA
from sklearn.exceptions import ConvergenceWarning
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest, chi2, RFE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils._testing import ignore_warnings

from sklearn import tree, naive_bayes

In [3]:
sklearn.__version__

'0.24.2'

# Предобработка данных

## Загрузка данных

### Классификация

In [4]:
df = pd.read_csv('../data/bank_additional_preprocessed.csv', sep=';')

In [5]:
df

,age,campaign,pdays,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,job_admin.,...,poutcome_nonexistent,poutcome_success,previous_0,previous_1,previous_2,previous_3,previous_4,previous_5,previous_6,previous_7
0,56,1,999,1.1,93.994,-36.4,4.857,5191.0,0,0,...,1,0,1,0,0,0,0,0,0,0
1,57,1,999,1.1,93.994,-36.4,4.857,5191.0,0,0,...,1,0,1,0,0,0,0,0,0,0
2,37,1,999,1.1,93.994,-36.4,4.857,5191.0,0,0,...,1,0,1,0,0,0,0,0,0,0
3,40,1,999,1.1,93.994,-36.4,4.857,5191.0,0,1,...,1,0,1,0,0,0,0,0,0,0
4,56,1,999,1.1,93.994,-36.4,4.857,5191.0,0,0,...,1,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,1,999,-1.1,94.767,-50.8,1.028,4963.6,1,0,...,1,0,1,0,0,0,0,0,0,0
41184,46,1,999,-1.1,94.767,-50.8,1.028,4963.6,0,0,...,1,0,1,0,0,0,0,0,0,0
41185,56,2,999,-1.1,94.767,-50.8,1.028,4963.6,0,0,...,1,0,1,0,0,0,0,0,0,0
41186,44,1,999,-1.1,94.767,-50.8,1.028,4963.6,1,0,...,1,0,1,0,0,0,0,0,0,0


Посмотрим на сбалансированность признаков.

In [6]:
labels, samples = np.unique(df.y, return_counts=True)
labels, samples

(array([0, 1], dtype=int64), array([36548,  4640], dtype=int64))

Как мы видим, данные несбалансированы: данных класса 0 приблизительно в 10 раз больше, чем данных класса 1.

При таком дисбалансе при downsampling потеряется большое число информации, поэтому выполним upsampling.

In [7]:
df_0 = df.loc[df.y == 0].reset_index()
df_1_copy = df.loc[df.y == 1].reset_index()
df_1 = df.loc[df.y == 1].reset_index()

i = len(df_1_copy)
while i < len(df_0):
    print('GET i: {}'.format(i))
    if i + len(df_1_copy) < len(df_0):
        df_1 = df_1.append(df_1_copy)
        i += len(df_1_copy)
    else:
        df_1 = df_1.append(df_1_copy.loc[i % len(df_1_copy) : (len(df_0) - 1) % len(df_1_copy)])
        i = len(df_1)

print('GET samples: {} label 0, {} label 1'.format(len(df_0), len(df_1)))
df_sample = pd.concat([df_0, df_1], ignore_index=True).reset_index()
print('Make shuffle dataframe')
df_sample = df_sample.loc[np.random.permutation(len(df_sample))]

GET i: 4640
GET i: 9280
GET i: 13920
GET i: 18560
GET i: 23200
GET i: 27840
GET i: 32480
GET samples: 36548 label 0, 36548 label 1
Make shuffle dataframe


In [8]:
X = df_sample.drop('y', axis=1)
y = df_sample.y
X.shape, y.shape

((73096, 71), (73096,))

Для удобства обработки разделим признаки на категориальные и вещественные

In [9]:
numeric_cols = np.array(['age', 'campaign', 'pdays', 'emp.var.rate', 'cons.price.idx', 
           'cons.conf.idx', 'euribor3m', 'nr.employed'])
X_numeric = X[numeric_cols]

In [10]:
categorical_cols = list(set(X.columns.values.tolist()) - set(numeric_cols))
X_categorical = X[categorical_cols]
for col in categorical_cols:
    X_categorical[col] = X_categorical[col].astype('string')

D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [11]:
print(X_categorical.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73096 entries, 34712 to 64201
Data columns (total 63 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   default_yes                    73096 non-null  string
 1   poutcome_failure               73096 non-null  string
 2   month_jul                      73096 non-null  string
 3   previous_4                     73096 non-null  string
 4   job_blue-collar                73096 non-null  string
 5   month_apr                      73096 non-null  string
 6   day_of_week_wed                73096 non-null  string
 7   previous_3                     73096 non-null  string
 8   job_services                   73096 non-null  string
 9   housing_unknown                73096 non-null  string
 10  day_of_week_tue                73096 non-null  string
 11  month_may                      73096 non-null  string
 12  job_student                    73096 non-null  string
 1

Разделим данные на обучение и тест

In [12]:
X_train, X_test, X_train_cat, X_test_cat, X_train_num, X_test_num, y_train, y_test = train_test_split(X, X_categorical, X_numeric, y, test_size=0.25)

Нормализируем значения

In [13]:
scaler = StandardScaler()
scaler.fit(X_numeric)

X_train_num_sc = scaler.transform(X_train_num)
X_test_num_sc = scaler.transform(X_test_num)
X_num_sc = scaler.transform(X_numeric)

Соединяем воедино все значения

In [14]:
X_train_transform = np.hstack((X_train_num_sc, X_train_cat))
X_test_transform = np.hstack((X_test_num_sc, X_test_cat))
X_transform = np.hstack((X_num_sc, X_categorical))

Для некоторых алгоритмов введено условие неотрицательности, реализуем его

In [15]:
scaler = MinMaxScaler()
scaler.fit(X_transform)
X_positive = scaler.transform(X_transform)
X_train_positive = scaler.transform(X_train_transform)
X_test_positive = scaler.transform(X_test_transform)

In [16]:
X_positive

array([[0.18518519, 0.        , 1.        , ..., 0.        , 1.        ,
        0.        ],
       [0.18518519, 0.14545455, 1.        , ..., 0.        , 0.        ,
        1.        ],
       [0.25925926, 0.01818182, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.2345679 , 0.03636364, 1.        , ..., 0.        , 0.        ,
        1.        ],
       [0.62962963, 0.01818182, 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.20987654, 0.        , 1.        , ..., 0.        , 1.        ,
        0.        ]])

## Отбор признаков

### 1. Одномерный отбор признаков

Признаки, имеющие наиболее выраженную взаимосвязь с целевой переменной, могут быть отобраны с помощью статистических критериев. Библиотека scikit-learn содержит класс [SelectKBest](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest), реализующий одномерный отбор признаков (univariate feature selection). Этот класс можно применять совместно с различными статистическими критериями для отбора заданного количества признаков.

In [17]:
# feature extraction
test = SelectKBest(score_func=chi2, k=4)
fit = test.fit(X_positive, y)

# summarize scores
np.set_printoptions(precision=3)
print(fit.scores_)
X_train_k_best = fit.transform(X_train_positive)
X_test_k_best = fit.transform(X_test_positive)

[9.148e+00 8.262e+01 7.183e+02 2.867e+03 3.906e+02 4.998e+01 4.929e+03
 2.506e+03 3.000e+00 1.412e+02 1.904e+02 2.010e+02 9.170e+02 7.688e+02
 5.314e+00 7.207e+02 1.916e+02 1.341e+00 9.854e+00 1.682e+03 8.672e+02
 1.394e+02 2.546e+01 6.854e+01 3.126e+03 4.780e+00 9.137e+03 1.679e+03
 1.516e+03 1.448e+03 7.414e+00 1.341e+00 1.000e+00 1.038e+02 7.043e+00
 1.242e+01 1.134e+03 9.762e+02 9.432e+02 8.923e+00 1.570e+01 1.418e+03
 2.040e+01 3.026e+00 1.255e+02 5.197e+01 3.380e+01 8.754e-01 2.476e+01
 8.298e+01 1.526e+01 1.825e+03 9.450e-03 7.916e+00 4.760e+02 2.107e+03
 2.957e+02 1.205e+01 3.009e+00 1.544e-02 2.416e-02 1.219e+03 7.085e+00
 5.562e+03 9.762e+02 1.950e+01 3.479e+02 3.440e+02 3.553e+02 6.626e+00
 6.719e+01]


### 2. Рекурсивное исключение признаков

Метод рекурсивного исключения признаков (recursive feature elimination, RFE) реализует следующий алгоритм: модель обучается на исходном наборе признаков и оценивает их значимость, затем исключается один или несколько наименее значимых признаков, модель обучается на оставшихся признаках, и так далее, пока не останется заданное количество лучших признаков. В документации scikit-learn вы можете подробнее прочитать о классе [RFE](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.RFE.html#sklearn.feature_selection.RFE).

В примере ниже метод RFE применяется в сочетании с логистической регрессией для отбора 4-х лучших признаков. Для совместного использования с RFE можно выбирать различные модели, важно лишь, чтобы они были достаточно эффективны и совместимы с RFE.

In [18]:
# feature extraction
test = RFE(tree.DecisionTreeClassifier(), n_features_to_select=4)
fit = test.fit(X_transform, y)

# summarize scores
print('Num features: {}'.format(fit.n_features_))
print('Selected Features: {}'.format(fit.support_))
print('Feature Ranking: {}'.format(fit.ranking_))
X_train_recursive_dtc = fit.transform(X_train_positive)
X_test_recursive_dtc = fit.transform(X_test_positive)

Num features: 4
Selected Features: [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False  True False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True  True  True]
Feature Ranking: [68 67 66 65 64 63 62 61 60 59 58 57 56 55 54 53 52 51 50 49 48 47 46 45
 44 43  1 13 12 11 10  9  8  7 15 17 19 21 23 25 27 29 31 33 35 37 39 41
 42 40 38 36 34 32 30 28 26 24 22 20 18 16 14  6  5  4  3  2  1  1  1]


### 3. Метод главных компонент

Метод главных компонент (principal component analysis, PCA) позволяет уменьшить размерность данных с помощью преобразования на основе линейной алгебры. Пользователь может задать требуемое количество измерений (главных компонент) в результирующих данных.

В примере ниже мы выделяем 3 главных компоненты с помощью PCA.

Подробная информация о классе [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) доступна в документации scikit-learn. Если вас заинтересовала математика PCA, обратитесь к статье в [Википедии](https://en.wikipedia.org/wiki/Principal_component_analysis).

In [19]:
# feature extraction
test = PCA(n_components=4)
fit = test.fit(X_transform, y)

X_train_pca = fit.transform(X_train_positive)
X_test_pca = fit.transform(X_test_positive)

### 4. Отбор на основе важности признаков

Ансамблевые алгоритмы на основе деревьев решений, такие как случайный лес (random forest), позволяют оценить важность признаков.

В представленном ниже примере мы обучаем классификатор ExtraTreesClassifier, чтобы с его помощью определить важность признаков. Подробнее о классе [ExtraTreesClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html) можно узнать из документации scikit-learn.

In [20]:
# feature extraction
test = ExtraTreesClassifier()
fit = test.fit(X_transform, y)
print(fit.feature_importances_)

[1.802e-02 1.400e-02 1.255e-02 2.646e-02 9.902e-03 8.928e-03 6.018e-02
 3.975e-02 4.933e-07 3.438e-03 2.409e-03 1.941e-04 4.655e-03 3.143e-03
 4.138e-03 4.887e-04 2.966e-03 9.455e-04 4.234e-03 1.380e-02 2.016e-03
 5.089e-03 4.545e-03 2.202e-03 7.858e-03 3.004e-04 5.549e-01 1.465e-03
 3.690e-03 4.468e-03 4.639e-03 9.256e-04 2.938e-05 2.278e-03 7.334e-05
 5.605e-03 1.218e-03 7.763e-03 3.122e-03 2.631e-03 1.793e-05 2.031e-03
 2.689e-03 4.191e-03 4.690e-03 1.940e-03 1.517e-03 4.167e-03 2.607e-03
 5.367e-05 2.063e-03 4.195e-03 2.754e-03 1.563e-03 5.700e-04 4.301e-02
 4.915e-03 5.633e-03 2.042e-03 8.498e-04 3.302e-03 8.039e-03 4.572e-03
 1.304e-02 8.048e-03 3.431e-03 5.654e-03 4.697e-03 3.804e-03 3.994e-03
 4.902e-03]


In [21]:
eps = 0.1
important = fit.feature_importances_ > eps
X_train_positive.shape, important.shape

((54822, 71), (71,))

In [22]:
important

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False])

In [23]:
X_train_etc = X_train_positive[:, important]
X_test_etc = X_test_positive[:, important]
X_train_etc.shape, X_train_positive.shape

((54822, 1), (54822, 71))

# Обучение модели

Напишем методы, которые будут обучать модель и предсказывать результат

In [24]:
def print_metrics(y_true, y_pred, is_classification=True):
    if is_classification:
        print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))

In [25]:
def fit_predict(X_train, X_test, y_train, y_test, 
                model=tree.DecisionTreeClassifier, init_parameters={}, 
                verbose=True, **parameters):
    print('Start fit model and predict values')
    classifier_model = model(**init_parameters)

    clf_model = GridSearchCV(classifier_model, parameters)
    clf_model.fit(X_train, y_train)
    # Get best model
    best_est = clf_model.best_estimator_
    if verbose:
        print('GridSearchCV: best estimator:\n{},\nwith best parameters:\n{}'.format(clf_model.best_estimator_, clf_model.best_params_))
    # Predict value
    y_pred = best_est.predict(X_test)
    if verbose:
        print_metrics(y_test, y_pred)

In [55]:
@ignore_warnings(category=ConvergenceWarning)
def print_with_selection_features(model=tree.DecisionTreeClassifier, init_parameters={}, **parameters):
    print('Начинаем сравнение разных методов отбора признаков.')
    print()
    print('Без отбора признаков')
    %time fit_predict(X_train_transform, X_test_transform, y_train, y_test, model=model, init_parameters=init_parameters, **parameters)
    print()
    print('Одномерный отбор признаков')
    %time fit_predict(X_train_k_best, X_test_k_best, y_train, y_test, model=model, init_parameters=init_parameters, **parameters)
    print()
    print('Рекурсивное исключение признаков')
    %time fit_predict(X_train_recursive_dtc, X_test_recursive_dtc, y_train, y_test, model=model, init_parameters=init_parameters, **parameters)
    print()
    print('Метод главных компонент')
    %time fit_predict(X_train_pca, X_test_pca, y_train, y_test, model=model, init_parameters=init_parameters, **parameters)
    print()
    print('Отбор на основе важности признаков')
    %time fit_predict(X_train_etc, X_test_etc, y_train, y_test, model=model, init_parameters=init_parameters, **parameters)

## [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

In [35]:
parameters_dtc = {
    'max_depth': [1, 3, 5, 10, 30],
    'min_samples_split': [2, 3, 5, 8]
}

In [37]:
print_with_selection_features(model=tree.DecisionTreeClassifier, **parameters_dtc)

Начинаем сравнение разных методов отбора признаков.

Без отбора признаков
Start fit model and predict values
GridSearchCV: best estimator:
DecisionTreeClassifier(max_depth=1),
with best parameters:
{'max_depth': 1, 'min_samples_split': 2}
[[9108    0]
 [   0 9166]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9108
           1       1.00      1.00      1.00      9166

    accuracy                           1.00     18274
   macro avg       1.00      1.00      1.00     18274
weighted avg       1.00      1.00      1.00     18274

Wall time: 1min 55s

Одномерный отбор признаков
Start fit model and predict values
GridSearchCV: best estimator:
DecisionTreeClassifier(max_depth=1),
with best parameters:
{'max_depth': 1, 'min_samples_split': 2}
[[9108    0]
 [   0 9166]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9108
           1       1.00      1.00      1.00      9166

   

Алгоритм работает идеально.

Попробуем взять другой алгоритм.

## [GaussianNB](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html)

Naive bayes algorithm

In [41]:
parameters_nb = {
    'var_smoothing': [1e-10, 1e-9, 1e-8, 1e-5, 1e-2, 1e-1, 1, 10, 100]
}

In [42]:
print_with_selection_features(model=naive_bayes.GaussianNB, **parameters_nb)

Начинаем сравнение разных методов отбора признаков.

Без отбора признаков
Start fit model and predict values
GridSearchCV: best estimator:
GaussianNB(var_smoothing=100),
with best parameters:
{'var_smoothing': 100}
[[8721  387]
 [ 498 8668]]
              precision    recall  f1-score   support

           0       0.95      0.96      0.95      9108
           1       0.96      0.95      0.95      9166

    accuracy                           0.95     18274
   macro avg       0.95      0.95      0.95     18274
weighted avg       0.95      0.95      0.95     18274

Wall time: 50.9 s

Одномерный отбор признаков
Start fit model and predict values
GridSearchCV: best estimator:
GaussianNB(var_smoothing=1e-10),
with best parameters:
{'var_smoothing': 1e-10}
[[8733  375]
 [ 776 8390]]
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      9108
           1       0.96      0.92      0.94      9166

    accuracy                           0.94     1

На данной моделе хорошо видно, что разные виды отборов приносят разные результаты.

В данной задаче отработали (по убыванию точности):
1. Отбор на основе важности признаков (100% точность)
1. Рекурсивное исключение признаков (Уменьшает число ошибок как в классе 1, так и в классе 0)
1. Без отбора
1. Одномерный отбор признаков (Сильно ошибается с классом 1, присваивая его классу 0, в классе 0 ошибок приблизительно так же, как если бы отбора не было)
1. Метод главных компонент (Очень много ошибок как в классе 0, так и в классе 1)

## [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [56]:
parameters_lr = {
    'class_weight': ['balanced'],
    'solver': ['liblinear', 'saga'],
    'penalty': ['l1', 'l2'],
}

In [57]:
print_with_selection_features(model=LogisticRegression, **parameters_lr)

Начинаем сравнение разных методов отбора признаков.

Без отбора признаков
Start fit model and predict values
GridSearchCV: best estimator:
LogisticRegression(class_weight='balanced', penalty='l1', solver='liblinear'),
with best parameters:
{'class_weight': 'balanced', 'penalty': 'l1', 'solver': 'liblinear'}
[[9108    0]
 [   0 9166]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      9108
           1       1.00      1.00      1.00      9166

    accuracy                           1.00     18274
   macro avg       1.00      1.00      1.00     18274
weighted avg       1.00      1.00      1.00     18274

Wall time: 1min 56s

Одномерный отбор признаков
Start fit model and predict values
GridSearchCV: best estimator:
LogisticRegression(class_weight='balanced', penalty='l1', solver='liblinear'),
with best parameters:
{'class_weight': 'balanced', 'penalty': 'l1', 'solver': 'liblinear'}
[[9108    0]
 [   0 9166]]
              precision    

Получаем очень интересные результаты.

Без отбора признаков, Одномерный отбор признаков и Отбор на основе важности признаков проявили точность 100%.

Рекурсивное исключение признаков практически не ошибается, но имеется мизерное число ошибок.

Метод главных компонент отработал ужасно на данных.

# Вывод

В данной работе мы разобрались с методами отбора признаков, рассмотрели некоторые из них и сравнили их между собой на своих данных.